# Analyse scène compétitive Pokemon

## Analyse exploratoire

### Création des bases des données

Commençons par récupérer les données d'utilisation des différents pokemon sur la scène compétitive 6v6. Le site Smogon propose une banque de données conséquente sur ces utilisations au sein des différentes catégories compétitives.

Nous allons utiliser la classe d'objet *Contact_Smogon* pour récupérer les fichiers bruts aux formats *.txt* avant de les convertir en *.csv*. Dans un premier temps nous regarderons en premier lieu le format 7G OverUsed tous niveaux, nous allons donc récupérer toutes les données correspondantes, à raison d'un fichier par mois.

In [1]:
#On importe la classe requise, qui gère déjà les imports nécessaires
import contact_smogon as cs

#On initialise avant de lancer la requête, par défaut elle demandera toutes les dates disponibles
contact = cs.Contact_Smogon(gen='gen7',tier='ou',rating='0')
contact.request_data()

Folder /raw_data succesfully created
File /rawdata_2016-11_gen7ou-0.txt succesfully created
Folder /clean_data succesfully created
File /data_2016-11_gen7ou-0.txt.csv succesfully created
File /rawdata_2016-12_gen7ou-0.txt succesfully created
File /data_2016-12_gen7ou-0.txt.csv succesfully created
File /rawdata_2017-02_gen7ou-0.txt succesfully created
File /data_2017-02_gen7ou-0.txt.csv succesfully created
File /rawdata_2017-03_gen7ou-0.txt succesfully created
File /data_2017-03_gen7ou-0.txt.csv succesfully created
File /rawdata_2017-04_gen7ou-0.txt succesfully created
File /data_2017-04_gen7ou-0.txt.csv succesfully created
File /rawdata_2017-05_gen7ou-0.txt succesfully created
File /data_2017-05_gen7ou-0.txt.csv succesfully created
File /rawdata_2017-06_gen7ou-0.txt succesfully created
File /data_2017-06_gen7ou-0.txt.csv succesfully created
File /rawdata_2017-07_gen7ou-0.txt succesfully created
File /data_2017-07_gen7ou-0.txt.csv succesfully created
File /rawdata_2017-08_gen7ou-0.txt s

File /rawdata_2023-02_gen7ou-0.txt succesfully created
File /data_2023-02_gen7ou-0.txt.csv succesfully created
File /rawdata_2023-03_gen7ou-0.txt succesfully created
File /data_2023-03_gen7ou-0.txt.csv succesfully created
File /rawdata_2023-04_gen7ou-0.txt succesfully created
File /data_2023-04_gen7ou-0.txt.csv succesfully created
File /rawdata_2023-05_gen7ou-0.txt succesfully created
File /data_2023-05_gen7ou-0.txt.csv succesfully created
File /rawdata_2023-06_gen7ou-0.txt succesfully created
File /data_2023-06_gen7ou-0.txt.csv succesfully created


Nous allons fusionner les bases de données que nous avons à disposition, à savoir :
- les bases d'utilisation des différents pokémon pour chaque mois d'existence de la compétition,
- la base du pokédex contenant les statistiques de chaque pokemon.

In [21]:
import pandas as pd
import os

#Les options suivantes permettent d'afficher toutes les lignes et colonnes voulues lorsqu'on appelle la fonction print
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

#On liste les colonnes du pokédex qui nous intéressent
columns = ['name', 'type1', 'type2','abilities', 'attack', 'defense', 'hp',
           'sp_attack', 'sp_defense', 'speed', 'generation', 'pokedex_number']

"""
resistances = ['against_bug', 'against_dark', 'against_dragon',
               'against_electric', 'against_fairy', 'against_fight', 'against_fire',
               'against_flying', 'against_ghost', 'against_grass', 'against_ground',
               'against_ice', 'against_normal', 'against_poison', 'against_psychic',
               'against_rock', 'against_steel', 'against_water']
"""

pokedex = pd.read_csv('pokemon_data/pokemon.csv')[columns]

dir_path = 'clean_data'
for path in os.listdir(dir_path):
    if os.path.isfile(os.path.join(dir_path, path)):
        temp_data = pd.read_csv(dir_path+'/'+path)
        #On récupère les données qui nous intéressent, à savoir la fréquence d'utilisation "usage_pct"
        date = path[5:12]
        temp_data.rename(columns={"pokemon" : "name", "usage_pct": date}, inplace=True)
        pokedex = pokedex.merge(temp_data[["name", date]], how='left', on='name')
        
pokedex.to_csv('database.csv', index_label='pokedex_number')

pokedex.head(9)

,name,type1,type2,abilities,attack,defense,hp,sp_attack,sp_defense,speed,generation,pokedex_number,2016-11,2016-12,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06
0,Bulbasaur,grass,poison,"['Overgrow', 'Chlorophyll']",49,49,45,65,65,45,1,1,0.00074,NaN,0.03461,0.02797,0.02382,0.02720,0.01682,0.01843,0.02595,0.03974,0.02511,0.02080,0.02069,0.02855,0.01648,0.02311,0.01988,0.01725,0.01794,0.02530,0.01559,0.01734,0.01399,0.03277,0.02829,0.02683,0.02314,0.02509,0.02230,0.01944,0.01551,0.02139,0.01650,0.02930,0.02045,0.02795,0.03789,0.03421,0.06302,0.05264,0.06056,0.02220,0.02383,0.04321,0.04417,0.03563,0.03771,0.02933,0.03967,0.02490,0.03527,0.02879,0.03378,0.03517,0.03760,0.03142,0.04389,0.05165,0.03082,0.03361,0.02527,0.03349,0.05667,0.01827,0.02346,0.01749,0.03871,0.01270,0.02207,0.02294,0.01961,0.03216,0.01220,0.01797,0.03031,0.02132,0.01944,0.03856,0.01575
1,Ivysaur,grass,poison,"['Overgrow', 'Chlorophyll']",62,63,60,80,80,60,1,2,0.00097,NaN,0.00877,0.01019,0.01065,0.01153,0.00646,0.01171,0.01033,0.01352,0.00891,0.01092,0.01086,0.00977,0.01123,0.01396,0.01542,0.01103,0.01656,0.01505,0.02054,0.01797,0.01732,0.02527,0.02354,0.02169,0.02175,0.01733,0.02847,0.02013,0.01734,0.01810,0.01273,0.02084,0.03239,0.04018,0.01329,0.02444,0.04198,0.05683,0.03729,0.01896,0.01197,0.01855,0.02084,0.01929,0.03040,0.01607,0.01510,0.02353,0.02183,0.04048,0.02217,0.02660,0.01288,0.01433,0.01507,0.01594,0.01158,0.02390,0.01316,0.03202,0.00558,0.05025,0.02307,0.03032,0.06745,0.02575,0.00910,0.00765,0.00792,0.00984,0.01017,0.00751,0.03697,0.01302,0.00499,0.00191,0.00463
2,Venusaur,grass,poison,"['Overgrow', 'Chlorophyll']",100,123,80,122,120,80,1,3,0.04425,NaN,0.99016,0.97451,0.89342,1.12268,1.02873,0.89334,0.92881,0.97994,0.97703,0.80170,0.78573,0.94346,1.00323,0.98483,0.98212,0.99975,1.07955,0.93776,0.91757,0.97033,1.13316,1.15743,1.07120,1.23751,1.25630,1.09219,1.06142,1.07328,1.05697,1.11976,1.17002,1.42118,1.49850,1.60336,1.83298,1.74389,1.89236,1.74093,1.79848,1.84136,1.77656,1.76807,1.81651,1.84293,1.99035,2.00136,1.89255,1.88023,1.63743,1.56226,1.57472,1.49181,1.74601,1.82667,1.62228,1.84764,1.76369,1.60150,1.63142,1.73750,1.72326,1.64481,1.34682,1.33558,1.50314,1.36292,1.55990,1.34951,1.43368,1.89932,1.85906,1.24737,1.23153,1.30983,1.23125,1.18794,1.43387
3,Charmander,fire,NaN,"['Blaze', 'Solar Power']",52,43,39,60,50,65,1,4,0.00051,NaN,0.01430,0.01304,0.01427,0.01086,0.01000,0.01514,0.01261,0.01057,0.00927,0.01097,0.00997,0.01325,0.00992,0.01472,0.00900,0.01034,0.00807,0.00828,0.00764,0.00666,0.00766,0.01697,0.00911,0.00626,0.00809,0.00769,0.00644,0.01105,0.00787,0.00762,0.00703,0.00696,0.00923,0.01120,0.01266,0.00911,0.02001,0.01635,0.01733,0.01353,0.00874,0.01688,0.01544,0.01351,0.01693,0.01739,0.02013,0.01259,0.00766,0.01568,0.01483,0.01086,0.01062,0.01091,0.01463,0.00781,0.01372,0.02195,0.00627,0.02486,0.02855,0.00933,0.00736,0.00972,0.00215,0.00688,0.00573,0.00401,0.00717,0.01444,0.00457,0.00536,0.03197,0.01191,0.00446,0.01258,0.01606
4,Charmeleon,fire,NaN,"['Blaze', 'Solar Power']",64,58,58,80,65,80,1,5,0.00046,NaN,0.00891,0.01304,0.01270,0.00729,0.00922,0.01323,0.01116,0.01453,0.01008,0.00871,0.01030,0.01036,0.00940,0.00930,0.01030,0.00619,0.00674,0.00762,0.00764,0.00542,0.00653,0.00915,0.01037,0.01412,0.00796,0.00632,0.00453,0.00946,0.00708,0.00562,0.00631,0.00564,0.00718,0.01010,0.01484,0.00889,0.01564,0.01232,0.01614,0.00721,0.00531,0.02110,0.00777,0.01400,0.01809,0.01032,0.0